In [10]:
import json
import numpy as np
import pandas as pd
import xarray as xr
from matplotlib import pyplot as plt

import add_path
import toolkit.allen_helpers.stimuli as st
import toolkit.pipeline.signal as ps
from toolkit.pipeline.data_io import SessionDirectory, FILES
from toolkit.pipeline.global_settings import GLOBAL_SETTINGS

pd.set_option('display.max_columns', None)

with open('config.json') as f:
    config = json.load(f)

## Get session and load data

In [2]:
session_id = config['session_id']
ecephys_structure_acronym = config['ecephys_structure_acronym']
print(f"Session ID: {session_id}")

Session ID: 766640955


In [ ]:
session_dir = SessionDirectory(session_id, ecephys_structure_acronym, cache_lfp=True)

probe_info = session_dir.load_probe_info()
if not session_dir.has_lfp_data:
    raise ValueError(f"Session {session_id} has no LFP data")

session = session_dir.session

preferred_orientation = session_dir.load_preferred_orientations().sel(
    layer=[GLOBAL_SETTINGS['layer_of_interest']]).values

# load bands of interest
bands_of_interest = FILES.load('bands_of_interest')
layer_of_interest = GLOBAL_SETTINGS['layer_of_interest']

c:\Users\chen8\.conda\envs\allen\lib\site-packages\hdmf\spec\namespace.py:590: UserWarning: Ignoring the following cached namespace(s) because another version is already loaded:
core - cached version: 2.2.2, loaded version: 2.7.0
The loaded extension(s) may not be compatible with the cached extension(s) in the file. Please check the extension documentation and ignore this warning if these versions are compatible.
  self.warn_for_ignored_namespaces(ignored_namespaces)


In [154]:
stimulus_presentations = session.stimulus_presentations
session_type = session.session_type

stimulus_names = st.STIMULUS_NAMES[session_type]
drifting_gratings_stimuli = st.STIMULUS_CATEGORIES[session_type]['drifting_gratings']
natural_movies_stimuli = st.STIMULUS_CATEGORIES[session_type]['natural_movies']

## Analyze data

### Get frequency bands

In [182]:
def find_band_in_layers(bands_ds, layer_of_interest):
    band = bands_ds.bands.sel(layer=layer_of_interest)
    if np.isnan(band).all():
        wave_band = bands_ds.wave_band.item().title()
        print(f"{wave_band} band not found in the layer of interest '{layer_of_interest}'. "
            "Trying to find them in other layers.")
        band = ps.get_band_with_highest_peak(bands_ds)
        if band is None:
            print(f"{wave_band} band not found in any layer.")
        else:
            print(f"{wave_band} band found in layer '{band.layer.item()}'.")
    return band

In [ ]:
wave_bands = ['beta', 'gamma']

if session_id in bands_of_interest.session_id:
    freq_band = bands_of_interest.sel(wave_band=wave_bands, session_id=session_id)
else:
    print("Warning: Bands of interest not found in the PSD of this session. "
        f"Trying to find them in layers other than the layer of interest '{layer_of_interest}'.")
    bands_ds = session_dir.load_wave_bands()
    beta_stim = drifting_gratings_stimuli[0] + '_filtered'
    gamma_stim = natural_movies_stimuli[0]
    beta_band = find_band_in_layers(bands_ds.sel(stimulus=beta_stim, wave_band='beta'), layer_of_interest)
    gamma_band = find_band_in_layers(bands_ds.sel(stimulus=gamma_stim, wave_band='gamma'), layer_of_interest)
    if beta_band is None or gamma_band is None:
        average_wave_bands = FILES.load('average_wave_bands', session_type=session_type, session_set='selected_sessions')
        if beta_band is None:
            print("Beta band not found in the session. Trying to find the average bands from selected sessions.")
            beta_band = find_band_in_layers(average_wave_bands.sel(stimulus=beta_stim, wave_band='beta'), layer_of_interest)
            if beta_band is None:
                raise ValueError("No beta band found.")
        if gamma_band is None:
            print("Gamma band not found in the session. Trying to find the average bands from selected sessions.")
            gamma_band = find_band_in_layers(average_wave_bands.sel(stimulus=gamma_stim, wave_band='gamma'), layer_of_interest)
            if gamma_band is None:
                raise ValueError("No gamma band found.")
    freq_band = xr.concat([beta_band, gamma_band], dim='wave_band')

### Process LFP

In [31]:
group_width = 1  # Number of channels to the left and right of the central channel
extend_time = 1.0  # extend time at the start and end of each block to avoid boundary effect for filtering
filter_orientation = False  # whether use only preferred orientation

In [6]:
lfp_groups, channel_groups = ps.get_lfp_channel_groups(session_dir,
    probe_info['central_channels'], probe_id=probe_info['probe_id'], width=group_width)

#### Drifting gratings LFP

In [18]:
stim = drifting_gratings_stimuli[0]  # first drifting grating stimulus
drifting_gratings_window = (-0.5, 0.5)  # right boundary < extend_time

In [19]:
stimulus_trials = st.get_stimulus_trials(stimulus_presentations, stim)
stimulus_blocks = st.get_stimulus_blocks(stimulus_trials)
conditions = st.presentation_conditions(stimulus_trials.presentations)

In [40]:
window = (drifting_gratings_window[0], stimulus_trials.duration + drifting_gratings_window[1])
aligned_lfp, valid_trials = st.align_trials(
    lfp_groups, stimulus_trials, window=window, ignore_nan_trials='any')

if valid_trials is not None:  # if any trial is dropped by NaN values
    cond_presentation_id = st.presentation_conditions(valid_trials.presentations)[1]
    if len(conditions[1]) != len(cond_presentation_id):
        diff = set(conditions[1].keys()) - set(cond_presentation_id.keys())
        raise ValueError(f"All trials are dropped by NaN values in {stim} for conditions: {diff}")
    conditions = (conditions[0], cond_presentation_id)

valid_conditions = ps.filter_conditions(conditions[0])
if filter_orientation:
    valid_conditions = valid_conditions.sel(orientation=preferred_orientation)
valid_presentations = np.sort(np.concatenate([conditions[1][c] for c in valid_conditions.values.ravel()]))
aligned_lfp = aligned_lfp.sel(presentation_id=valid_presentations)

In [42]:
aligned_lfp

<xarray.DataArray 'LFP' (layer: 6, presentation_id: 300,
                         time_from_presentation_onset: 3753)>
array([[[-8.25500107e+00, -7.66999960e+00, -9.09999943e+00, ...,
         -5.00499964e+00, -7.60499954e+00, -1.63150005e+01],
        [-7.99499941e+00, -6.88999987e+00, -1.03350000e+01, ...,
         -1.53399992e+01, -8.31999969e+00, -1.39099998e+01],
        [-1.74199982e+01, -1.30000000e+01, -8.44999981e+00, ...,
          3.12000012e+00,  2.07999969e+00,  3.38000011e+00],
        ...,
        [-5.84999990e+00, -1.20250006e+01, -1.50799990e+01, ...,
         -4.55000019e+00, -6.11000013e+00,  2.99000001e+00],
        [ 1.57950010e+01,  1.74850006e+01,  4.80999994e+00, ...,
          1.02049999e+01,  8.06000042e+00, -3.24999958e-01],
        [ 9.75000083e-01,  2.85999990e+00, -2.79500008e+00, ...,
         -1.22850008e+01, -7.86499929e+00, -2.12549992e+01]],

       [[-2.15474991e+02, -1.65555008e+02, -1.52294998e+02, ...,
         -7.52049942e+01, -1.06860008e+02, -1.32599991e+02],
        [-1.25774994e+02, -1.60029984e+02, -1.73744995e+02, ...,
         -2.04425003e+02, -1.64255005e+02, -1.35590012e+02],
        [-1.24735008e+02, -1.19795006e+02, -1.01790001e+02, ...,
         -1.84340012e+02, -2.09364990e+02, -1.87005005e+02],
...
          2.17749996e+01,  1.23499985e+01,  2.88599987e+01],
        [ 3.23699989e+01,  3.84149971e+01,  3.13949986e+01, ...,
         -5.26499977e+01, -7.99500046e+01, -8.74899979e+01],
        [-7.09800034e+01, -6.20749969e+01, -5.46649971e+01, ...,
         -3.78299980e+01, -6.24649963e+01, -6.33099976e+01]],

       [[-1.23109993e+02, -1.21355003e+02, -1.16739998e+02, ...,
         -8.96999893e+01, -1.08745003e+02, -1.25449989e+02],
        [-1.29284988e+02, -1.58925003e+02, -1.66595001e+02, ...,
          4.47849998e+01,  4.56950035e+01,  3.49049988e+01],
        [ 3.13299999e+01,  2.43099976e+01,  1.55349998e+01, ...,
         -2.86000004e+01, -3.36699982e+01, -4.17949982e+01],
        ...,
        [ 2.11900005e+01,  1.72900009e+01, -2.59999967e+00, ...,
          5.54450035e+01,  5.62899971e+01,  6.07749939e+01],
        [ 2.31399994e+01,  2.55450001e+01,  2.78199997e+01, ...,
         -4.60850029e+01, -6.87049942e+01, -6.90299988e+01],
        [-5.57049980e+01, -5.77849998e+01, -4.98549995e+01, ...,
          1.29999924e+00, -2.54149990e+01, -3.41900024e+01]]],
      dtype=float32)
Coordinates:
  * presentation_id               (presentation_id) int64 40339 40340 ... 40938
  * time_from_presentation_onset  (time_from_presentation_onset) float64 -0.5...
  * layer                         (layer) <U3 '1' '2/3' '4' '5' '6a' '6b'
Attributes:
    fs:       1249.99984068252
    unit:     uV